In [1]:
import pandas as pd
may_trip = pd.read_csv('C:/Users/6106057/nss-data-analytics/projects/nashville-scooters-scooter-snatchers/data/may_trip.csv')
may_trip.head(10)

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt
0,2019-05-01 00:00:55.423000,Bird,BRD2134,Powered9EAJL,3.000000,958.00528,2019-05-01 00:00:00,00:00:20.460000,2019-05-01 00:00:00,00:02:52.346666,36.15710,-86.80360,36.15660,-86.80670,"[[36.157235,-86.803612],[36.157235,-86.80362],...",2019-05-02 05:30:23.780000
1,2019-05-01 00:03:33.147000,Lyft,LFT5,Powered296631,1.715600,1371.39112,2019-05-01 00:00:00,00:01:50.090000,2019-05-01 00:00:00,00:03:33.026666,36.15797,-86.77896,36.16054,-86.77689,"[[36.15797,-86.77896],[36.15795,-86.77873],[36...",2019-05-02 07:20:32.757000
2,2019-05-01 00:05:55.570000,Bird,BRD2168,Powered7S2UU,3.000000,2296.58800,2019-05-01 00:00:00,00:03:47.363333,2019-05-01 00:00:00,00:07:13.596666,36.15470,-86.78180,36.15650,-86.78680,"[[36.155068,-86.782124],[36.156597,-86.78675]]",2019-05-02 05:30:24.530000
3,2019-05-01 00:05:55.570000,Bird,BRD2166,PoweredZIIVX,3.000000,1200.78744,2019-05-01 00:00:00,00:04:21.386666,2019-05-01 00:00:00,00:06:59.176666,36.14940,-86.77950,36.15310,-86.77960,"[[36.149741,-86.779344],[36.149741,-86.779327]...",2019-05-02 05:30:24.237000
4,2019-05-01 00:05:55.570000,Bird,BRD2165,PoweredJ7MB3,2.000000,351.04988,2019-05-01 00:00:00,00:04:27.796666,2019-05-01 00:00:00,00:06:23.150000,36.17780,-86.78660,36.17740,-86.78760,"[[36.177699,-86.786477],[36.177711,-86.786469]...",2019-05-02 05:30:24.207000
5,2019-05-01 00:05:55.570000,Bird,BRD2167,PoweredEJF1K,3.000000,984.25200,2019-05-01 00:00:00,00:03:56.546666,2019-05-01 00:00:00,00:06:45.786666,36.15440,-86.78620,36.15150,-86.78440,"[[36.154368,-86.785909],[36.151689,-86.784331]...",2019-05-02 05:30:24.317000
6,2019-05-01 00:07:55.880000,Lyft,LFT10,Powered329418,1.093917,0.00000,2019-05-01 00:00:00,00:06:50.106666,2019-05-01 00:00:00,00:07:55.740000,36.15168,-86.78408,36.15171,-86.78411,"[[36.15168,-86.78408],[36.15171,-86.78411]]",2019-05-02 07:20:32.890000
7,2019-05-01 00:10:50.240000,Lyft,LFT13,Powered438572,2.954383,1161.41736,2019-05-01 00:00:00,00:07:52.853333,2019-05-01 00:00:00,00:10:50.116666,36.15193,-86.78439,36.15463,-86.78492,"[[36.15193,-86.78439],[36.15188,-86.78432],[36...",2019-05-02 07:20:32.990000
8,2019-05-01 00:10:56.013000,Bird,BRD2193,PoweredSX2AW,6.000000,2624.67200,2019-05-01 00:00:00,00:06:34.436666,2019-05-01 00:00:00,00:12:48.676666,36.17080,-86.78930,36.17460,-86.78600,"[[36.170995,-86.789593],[36.173193,-86.791207]...",2019-05-02 05:30:24.833000
9,2019-05-01 00:10:56.013000,Bird,BRD2194,PoweredL968V,6.000000,2624.67200,2019-05-01 00:00:00,00:05:36.870000,2019-05-01 00:00:00,00:11:19.183333,36.15650,-86.79710,36.15120,-86.79680,"[[36.15654,-86.797137],[36.153065,-86.798047],...",2019-05-02 05:30:24.920000


In [6]:
june_trip = pd.read_csv('C:/Users/6106057/nss-data-analytics/projects/nashville-scooters-scooter-snatchers/data/june_trip.csv')
june_trip.shape

(205627, 16)

In [7]:
july_trip = pd.read_csv('C:/Users/6106057/nss-data-analytics/projects/nashville-scooters-scooter-snatchers/data/july_trip.csv')
july_trip.shape

(134926, 16)